Our next step will be using tf-idf vectors to vectorize full text in our news pickled file containing a author. once it is vectorized just nearest neighbors to predict closest news stories to input text and return the author names that did it. 

The practical application of this include that if you have a kickstarter campaign going and are trying to reach out to jounos to spread the word, it would be nice to be able to shortlist the names of relevant journos who wrote a similar story to your campaign pitch. 

use sklearn to start out with, and by tuesday/webnesday we will make it interesting and switch to approximate nearest neighbor approaches. 

In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import ftfy

In [2]:
with open('../Data/news.pkl', 'rb') as f:
    authors = pickle.load(f)
    
with open('../Data/pickled_df.pkl', 'rb') as f:
    articles = pickle.load(f)
    
data = authors.merge(articles, on = 'title')[['author_x', 'article_count', 'site_name_x', 'topic', 'date',
                                              'url', 'title', 'title_sentiment', 'description', 'full_text']]
data = data.rename(columns = {'author_x': 'author', 'site_name_x': 'site_name'})

In [3]:
articles

,author,crawl_date,crawl_desc,crawl_url,date,description,full_text,img_url,site_name,title,topic,url,webpage_source,lang
4,"[""Liz Josaitis, Michigan State University Exte...",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The Food and Drug Administration has offered a...,A few examples include the offering of anti-a...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...,westernjournal.com,en
6,"[""Aja Styles""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,https://static.ffx.io/images/$zoom_0.3431%2C$m...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,brisbanetimes.com.au,en
11,"[""Jake Johnson""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,https://truthout.org/wp-content/uploads/2020/1...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,truthout.org,en
12,"[""Christine Favocci""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...,westernjournal.com,en
21,"[""Igor Derysh""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Fox aired segments debunking the claims after ...,Fox News and Fox Business just aired multiple...,https://truthout.org/wp-content/uploads/2020/1...,Truthout,Fox News Forced to Debunk Its Own False Voting...,business,https://truthout.org/articles/fox-news-forced-...,truthout.org,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43846,"[""Vanguard""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,"“There has never been a solid, unified and det...",The challenge of achieving racial justice in ...,https://i0.wp.com/www.vanguardngr.com/wp-conte...,Vanguard News,The challenge of achieving racial justice in A...,entertainment,https://www.vanguardngr.com/2020/12/the-challe...,vanguardngr.com,en
43851,"[""Vanguard""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,The United States on Friday imposed new sancti...,US imposes visa restrictions on Chinese offic...,https://i2.wp.com/www.vanguardngr.com/wp-conte...,Vanguard News,US imposes visa restrictions on Chinese offici...,business,https://www.vanguardngr.com/2020/12/us-imposes...,vanguardngr.com,en
43854,"[""Guto Llewelyn""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,Fan columnist Guto Llewelyn examines the situa...,When you subscribe we will use the informatio...,https://i2-prod.walesonline.co.uk/incoming/art...,WalesOnline,The reality of Swansea City's striker concerns...,sport,https://www.walesonline.co.uk/sport/football/t...,walesonline.co.uk,en
43860,"[""Laura McAllister""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,"If successful, I’d be the first woman from the...",Former Wales international Laura McAllister h...,https://i2-prod.walesonline.co.uk/incoming/art...,WalesOnline,"FIFA, football and Wales: Why I am standing fo...",sport,https://www.walesonline.co.uk/sport/sport-opin...,walesonline.co.uk,en


In [4]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [5]:
def clean_author(author):
    author = author.lower()
    author = re.sub(r'\|.*$', '', author) # remove everything after |
    author = re.sub(r'\,.*$', '', author) # remove evrything after ,
    punctuations = '''!()[]{};:'"\,<>./?#$%^&*_~'''
    for x in author.lower(): 
        if x in punctuations: author = author.replace(x, "") # remove punctuations except hyphen
    author = re.sub(r'(\d+)', '', author) # remove numbers
    author = re.sub(r'(--)', '', author)
    author = re.sub(r'(\w+@\w+)', '', author) # remove emails
    author = ' '.join([i.strip() for i in author.split()]) # remove spaces and lowercase
    author = re.sub(r'(\s+)(and)(\s+)(\w*)(\s*)(\w*)', '', author) # remove anything after and
    keywords = ['sa', 'tulsa', 'editor', 'writer', 'world-herald', 'news', 'world', 'richmond', 'times-dispatch', 'new',
                'hampshire', 'union', 'leader', 'for', 'the', 'state', 'journal', 'correspondant', 'sfgate', 'special',
                'from', 'the', 'gazette', 'times', 'staff', 'senior', 'dr', 'correspondent', 'by', 'editorial board',
                'research', 'wire reports', 'security', 'real', 'estate', 'to', 'post', 'and', 'courier', 'policy',
                'commercial', 'bureau', 'political', 'roanoke', 'college', 'football', 'editorial','democrat-gazette',
                'arizona', 'daily', 'star', '--hamburg', 'column', 'lincoln', 'managing', 'backstage', 'with', 'sports',
                'ii', 'iii', 'capitol', 'media', 'services']
    author = ' '.join([i for i in author.split() if i not in keywords]) # remove other keywords
    author = re.sub(r'(@)', '', author) # remove handles
    return author

In [6]:
data['clean_author'] = data.apply(lambda x: clean_author(x['author']), axis=1)

In [7]:
data

,author,article_count,site_name,topic,date,url,title,title_sentiment,description,full_text,clean_author
0,Aja Styles,14.0,Brisbane Times,entertainment,2020-12-23,https://www.brisbanetimes.com.au/national/west...,'Pack Lego': Perth family caught in hard borde...,-9.09,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,aja styles
1,Jake Johnson,33.0,Truthout,politics,2020-12-23,https://truthout.org/articles/congress-passes-...,Congress Passes COVID Relief With Billions in ...,18.18,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,jake johnson
2,Christine Favocci,19.0,The Western Journal,tech,2020-12-23,https://www.westernjournal.com/pa-man-facing-c...,PA Man Facing Charges of Unlawful Voting After...,-38.46,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,christine favocci
3,William Rivers Pitt,14.0,Truthout,politics,2020-12-23,https://truthout.org/articles/what-will-trump-...,What Will Trump Attempt in His Last Days? We M...,0.00,What Trump may do in his waning days is only u...,"The endgame being played out by Donald Trump,...",william rivers pitt
4,Amy Goodman,19.0,Truthout,business,2020-12-23,https://truthout.org/video/the-insufficient-co...,The Insufficient COVID Stimulus Must Not Be Fo...,-20.00,Critics say the $900 billion relief package do...,As Congress passes a $900 billion coronavirus...,amy goodman
...,...,...,...,...,...,...,...,...,...,...,...
119957,Olivia Tobin,56.0,Liverpool Echo,tech,2020-12-05,https://www.liverpoolecho.co.uk/news/liverpool...,"Boy, five, battling rare brain cancer will be ...",4.55,\n Five-year-old Aaron Wharton had surgery at ...,When you subscribe we will use the informatio...,olivia tobin
119958,Victoria Jones,92.0,WalesOnline,tech,2020-12-05,https://www.walesonline.co.uk/news/uk-news/how...,How to teach saving and spending to kids as yo...,0.00,Now might be a perfect time to involve childre...,When you subscribe we will use the informatio...,victoria jones
119959,Victoria Jones,92.0,WalesOnline,tech,2020-12-05,https://www.walesonline.co.uk/news/uk-news/spa...,Space experiment could unlock resources for mi...,0.00,Experimenting on the ISS allows scientists to ...,When you subscribe we will use the informatio...,victoria jones
119960,Nisha Mal,66.0,WalesOnline,tech,2020-12-05,https://www.walesonline.co.uk/news/uk-news/wom...,Woman's home is in Tier 2 while her garden fal...,-10.00,"'It's all one big conundrum,' says Sheila Herbert",Woman's home is in Tier 2 while her garden fa...,nisha mal


In [8]:
# Cleaning the text sentences so that punctuation marks, stop words &amp; digits are removed
def clean(doc):
    doc = ftfy.fix_text(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","", normalized)
    y = processed.split()
    return ' '.join(y)

In [9]:
data['text'] = ((data['title'] + ' ') * 4) + ((data['description'] + ' ') * 2) + (data['full_text'])

In [10]:
trial = data[:]
print(trial.shape)

(119962, 12)


In [11]:
trial['clean_text'] = trial.apply(lambda x: clean(x['text']), axis=1)

C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
trial.clean_text[0]

'pack lego perth family caught hard border crossfire christmas pack lego perth family caught hard border crossfire christmas pack lego perth family caught hard border crossfire christmas pack lego perth family caught hard border crossfire christmas perth mother clare found mostly confined small sydney city apartment sixyearold twoyearold northern beach outbreak take toll perth mother clare found mostly confined small sydney city apartment sixyearold twoyearold northern beach outbreak take toll perth mother clare found mostly confined small sydney city apartment two young daughter northern beach outbreak take toll clare flew cora georgia sydney december could reunite family christmas husband charles working remotely past year perth mother clare daughter cora sydney christmas hard border closure it pretty tough were really small apartment set small kid clare said we worked theory itd fine wed day tourist thing sydney obviously weve tried stay home much can apart grocery shop advertisemen

In [13]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(list(trial.clean_text))

In [14]:
vectorizer.get_feature_names()

['aa',
 'aaa',
 'aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaaaa',
 'aaaaaaallll',
 'aaaaahhhh',
 'aaaaand',
 'aaaand',
 'aaaargh',
 'aaaauuuuggghhh',
 'aaahhstyle',
 'aaarated',
 'aaaron',
 'aaastable',
 'aabritish',
 'aac',
 'aacc',
 'aachen',
 'aachhe',
 'aacps',
 'aacs',
 'aacspca',
 'aacta',
 'aactas',
 'aadhaar',
 'aadhaarlike',
 'aadhar',
 'aadhi',
 'aadi',
 'aadil',
 'aadis',
 'aaditi',
 'aaditya',
 'aadmi',
 'aadvantage',
 'aae',
 'aaegi',
 'aafje',
 'aafs',
 'aag',
 'aage',
 'aahana',
 'aahat',
 'aahhhhhh',
 'aahs',
 'aai',
 'aaib',
 'aail',
 'aails',
 'aaioperated',
 'aais',
 'aaja',
 'aajbharatbandhhai',
 'aajkiladki',
 'aakanksha',
 'aakash',
 'aal',
 'aala',
 'aalal',
 'aalam',
 'aaliyah',
 'aalla',
 'aallbig',
 'aalo',
 'aalsmeer',
 'aam',
 'aamar',
 'aamardeep',
 'aamc',
 'aamer',
 'aami',
 'aaminah',
 'aamir',
 'aampe',
 'aanan',
 'aanchal',
 'aandolan',
 'aang',
 'aanholt',
 'aaon',
 'aap',
 'aapan',
 'aapi',
 'aapke',
 'aapko',
 'aapl',
 'aapla',
 'aapled',
 'aapls',
 'aappa

In [15]:
nn = NearestNeighbors(n_neighbors=5, radius=1.0, algorithm='brute', leaf_size=30, metric='cosine', p=2, metric_params=None, n_jobs=None)
nn.fit(X)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [16]:
pickle.dump(nn, open('../Data/knn_model.p','wb'))

In [17]:
pickle.dump(vectorizer, open("../Data/knn_vectorizer.p", "wb"))

In [18]:
data.to_pickle('../Data/articles.pkl')